In [39]:
import os, sys
import pandas as pd
import numpy as np
import re
import nltk

from nltk.probability import *
from nltk import NaiveBayesClassifier
from nltk.tokenize import wordpunct_tokenize
from collections import defaultdict
#------------------------------------load directory and clean useless files--------------------------------------------------------------------
def getFilesFromDir (path):
    dir_content = os.listdir(path)
    dir_clean = filter(lambda x: (".DS_Store" not in x) and ("cmds" not in x), dir_content)
    
    msg = ["" for x in range(len(dir_clean))]
    for index in range (len(dir_clean)):
        file_path = path + '/' + dir_clean[index]
        msg[index] = getMessage(file_path)
    return msg
#------------------------------------load data in each file in the directory--------------------
def getMessage (path):
    fo = open (path)
    lines = fo.readlines()
    fo.close()
        
    for index in range (len(lines)):
        if (index <= (len(lines)-2)):
            if (('\n' in lines[index]) and (lines[index + 1] == '\n')):
                lines[index:index+2] = [''.join(lines[index:index+2])]
    #move to the first line break of the email which indicates the message body
    start_position = next(x for x in lines if '\n\n' in x)
    start_position_index = lines.index(start_position)
    lines_clean = lines[start_position_index + 1: len(lines)]
    #convert list into string
    str_msg = ''.join(lines_clean)
    return str_msg

amountOfSamplesPerSet = 500
amountOfFeaturesPerSet = 100
#file path
base_path = "/Users/yuliang/Downloads/naive_bayes"
ham_path = base_path + "/easy_ham"
ham_path2 = base_path + "/easy_ham_2"
spam_path = base_path + "/spam"
spam_path2 = base_path + "/spam_2"
#get files from directory
hamTrainDir = getFilesFromDir(ham_path)
hamTrainDir = hamTrainDir[:amountOfSamplesPerSet]
hamTestDir = getFilesFromDir(ham_path2)
spamTrainDir = getFilesFromDir(spam_path)
spamTrainDir = spamTrainDir[:amountOfSamplesPerSet]
spamTestDir = getFilesFromDir(spam_path2)
#-------------------------get message words from data------------------------------------------
def getMessageWords(file_msg, stopwords = []):
    file_msg = ''.join(file_msg)
    file_msg = re.sub('3D', '', file_msg)
    file_msg = re.sub(r'([^\s\w]|_)+', '', file_msg)
    
    file_msg_words = wordpunct_tokenize(file_msg.replace('=\n', '').lower())
    file_msg_words = filter(lambda x: x not in stopwords, file_msg_words)
    file_msg_words = [w for w in file_msg_words if re.search('[a-zA-Z]', w) and len(w) > 1]
    return file_msg_words

def getMessageWords2(file_msg, stopwords = []):
    file_msg = re.sub('3D', '', file_msg)
    file_msg = re.sub(r'([^\s\w]|_)+', '', file_msg)
    
    file_msg_words = wordpunct_tokenize(file_msg.replace('=\n', '').lower())
    file_msg_words = filter(lambda x: x not in stopwords, file_msg_words)
    file_msg_words = [w for w in file_msg_words if re.search('[a-zA-Z]', w) and len(w) > 1]
    return file_msg_words
#-------------------------get stop words--------------------------------------------------------
def getStopWords (path):
    fo = open (path)
    lines = fo.readlines()
    lines_clean = map(lambda x: str.replace(x, '\n', ''), lines)
    fo.close()
    return lines_clean
stop_words_given = getStopWords ("/Users/yuliang/Downloads/naive_bayes/stopwords.txt")
#-----------------------------------Term Document Matrix---------------------------------------------------------------
#get features
def getFeatures(file_msg, **kwargs):
    file_msg_words = getMessageWords(file_msg, **kwargs)
    words_list = nltk.FreqDist(file_msg_words)
    words_list_common = words_list.most_common()
    topFeatures = ["" for x in range(amountOfFeaturesPerSet)]
    for index in range(amountOfFeaturesPerSet):
        topFeatures[index]= words_list_common[index][0]
    return topFeatures

hamFeatures = getFeatures(hamTrainDir, stopwords = stop_words_given)
spamFeatures = getFeatures(spamTrainDir, stopwords = stop_words_given)
allFeatures = set(hamFeatures + spamFeatures)
allFeatures = list(allFeatures)
#print(trainFeatures)

def getFeaturesLabel(file_msg, label, allFeature, feature_extractor, **kwargs):
    features_label = []
    for w in file_msg:
        features = feature_extractor(w, allFeature, **kwargs)
        features_label.append((features, label))
    return features_label

def wordsIndicator(file_msg, allFeature, **kwargs):
    file_msg_words = getMessageWords2(file_msg, **kwargs)
    featureWords = allFeature
    features_dict = defaultdict(list)
    
    for w in file_msg_words:
        if w in allFeature:
            features_dict[w] = True
    return features_dict
#------------------------------------Train and test dataset---------------------------------------------
hamTrainFeature = getFeaturesLabel(hamTrainDir, 'ham', allFeatures, wordsIndicator, stopwords = stop_words_given)
spamTrainFeature = getFeaturesLabel(spamTrainDir, 'spam', allFeatures, wordsIndicator, stopwords = stop_words_given)
trainFeature = hamTrainFeature + spamTrainFeature
hamTestFeature = getFeaturesLabel(hamTestDir, 'ham', allFeatures, wordsIndicator, stopwords = stop_words_given)
spamTestFeature = getFeaturesLabel(spamTestDir, 'spam', allFeatures, wordsIndicator, stopwords = stop_words_given)
#------------------------------------Naive Bayes Classifier----------------------------------------------  
# Train the naive bayes classifier
naive_bayes_classifier = NaiveBayesClassifier.train(trainFeature)
print ('Test Spam accuracy: %.4f' %nltk.classify.accuracy(naive_bayes_classifier, hamTestFeature))
print ('Test Ham accuracy: %.4f' %nltk.classify.accuracy(naive_bayes_classifier, spamTestFeature))


Test Spam accuracy: 0.8507
Test Ham accuracy: 0.9792
